In [5]:
import glob
import subprocess
import os
import sys
import numpy as np
import pandas as pd
import re
from collections import defaultdict
import pathlib




In [52]:
def trodes_extract(trodes_directory, data, mode='-raw'):
    """
    This automates Trodes extraction by running a sequence of commands through
    the terminal. Trodes application is needed to be installed into the computer
    to run this function. Extraction folders will be generated within each .rec 
    data folder per recording. 
    Progress reports are printed: 'x out of n files extracted' after each extraction

    -raw file is needed to obtain timestamps from the ephys recording file. 
    -dio and -analogio are needed for ECU time stamps

    Args(3 total, 2 required):
        trodes_directory: str, directory to the trodes application folder 
        data: str, directory to your Trodes data folder 
              all files within data folder should be the .rec folders autogenerated
              by trodes extractions
        mode: str, default='-raw', a Trodes setting specifying which types 
              of files you want extracted. must be written as '-mode -mode'
              for each mode desired.
            Modes as described by Trodes: 
                -spikes: Spike waveform export (spike snippets only).

                -lfp: Continuous LFP band export.

                -spikeband: Continuous spike band export.

                -raw: Continuous raw band export.

                -stim: Continuous stimulation band export for stimulation capable recording channels.

                -dio: Digital IO channel state change export.

                -analogio: Continuous analog IO export.

                -mountainsort: One mountainsort file per ntrode (using raw band data)

                -kilosort: One kilosort file per ntrode (using raw band data).
    
    Returns: 
        none
"""
    data_path = data + '\*'
    folders = glob.glob(data_path)
    files = [glob.glob(f'{folder}\*merged.rec') for folder in folders]
    commands = []
    for i, folder in enumerate(folders):
        if files[i]:
            for file in files[i]:
                command =  'trodesexport '+ mode + ' -rec '+ file
                commands.append(command)
    no_commands = len(commands)
    i = 0
    for command in commands:
        subprocess.Popen(command, shell=True, cwd=trodes_directory).wait()
        i +=1
        print(i, 'out of', len(commands), 'files extracted')

def parse_fields(field_str):
    """
    Parses a string of fields into a numpy data type object.

    The input string should be formatted as '<fieldname num*type>' or 
    '<fieldname type>'. This function parses the string, extracts the field 
    names and data types, and creates a numpy data type object which can be 
    used to read data from a binary file.

    Args:
        field_str (str): The string specifying the fields.

    Returns:
        np.dtype: A numpy data type object that describes the structure of 
                  the data.

    Raises:
        SystemExit: If the provided field type is not a valid numpy data type.
    """
    
    # Clean up the string and split it into components
    components = re.split('\s', re.sub(r"\>\<|\>|\<", ' ', field_str).strip())
    
    dtype_spec = []  # Will hold tuples to specify the numpy data type
    
    # Iterate over pairs of components (field name and type)
    for i in range(0, len(components), 2):
        field_name = components[i]
        
        # Default values
        repeat_count = 1
        field_type_str = 'uint32'
        
        # If the field type string contains a '*', it indicates a repeat count
        if '*' in components[i+1]:
            split_types = re.split('\*', components[i+1])
            # Handle both 'num*type' and 'type*num'
            field_type_str = split_types[split_types[0].isdigit()]
            repeat_count = int(split_types[split_types[1].isdigit()])
        else:
            field_type_str = components[i+1]
        
        # Convert the field type string to an actual numpy data type
        try:
            field_type = getattr(np, field_type_str)
        except AttributeError:
            print(f"{field_type_str} is not a valid field type.")
            exit(1)
        else:
            dtype_spec.append((str(field_name), field_type, repeat_count))
    
    return np.dtype(dtype_spec)

def read_trodes_extracted_data_file(filename):
    """
    Reads the content of a Trodes extracted data file.

    This function opens a Trodes file, reads the settings, parses them into a dictionary, 
    and then reads the remaining data in the file as a numpy array according to the 
    data types specified in the settings. If the settings block does not start correctly,
    it raises an Exception.

    Args:
        filename (str): The path to the Trodes file to be read.

    Returns:
        dict: A dictionary where keys are settings field names and values are the 
              corresponding setting values. The actual data from the file is stored 
              under the 'data' key as a numpy array.

    Raises:
        Exception: If the settings block in the file does not start with '<Start settings>'.
    """

    with open(filename, 'rb') as f:
        # The first line of the file should start the settings block
        if f.readline().decode('ascii').strip() != '<Start settings>':
            raise Exception("Settings format not supported")
        
        # Flag indicating we're reading the settings block
        fields = True
        # Dictionary to hold the settings fields and values
        fields_text = {}
        
        # Iterate over the lines in the file
        for line in f:
            # If we're still reading the settings block
            if fields:
                line = line.decode('ascii').strip()
                # If we've not reached the end of the settings block, continue reading fields
                if line != '<End settings>':
                    key, value = line.split(': ')
                    fields_text.update({key.lower(): value})
                # If we've reached the end of the settings block, stop reading fields
                else:
                    fields = False
                    # Parse the 'fields' setting to get the data type
                    dt = parse_fields(fields_text['fields'])
                    fields_text['data'] = np.zeros([1], dtype = dt)
                    break
        
        # Read the remaining data from the file using the parsed data type
        dt = parse_fields(fields_text['fields'])
        data = np.fromfile(f, dt)
        fields_text.update({'data': data})
        return fields_text
            
def organize_single_trodes_export(dir_path, skip_raw_group0=True):
    """
    Organizes Trodes data files in a given directory. The data is stored in a dictionary. 
    The key is the penultimate (second to last) part of the file name (i.e., the part before the last dot in the file name). 
    The values in the dictionary are the parsed data from the Trodes files.

    Args:
        dir_path (str): The path to the directory containing the Trodes files.
        skip_raw_group0(bool): To skip the "raw_group0" file which contains the raw signal which uses a lot of memory
    Returns:
        dict: A dictionary with organized Trodes file data.
    """
    # Initialize dictionary to store results
    result = {}
    
    # Iterate over all files in the directory
    for file_name in os.listdir(dir_path):

        if skip_raw_group0 and "raw_group0" in file_name:
            continue
        # Attempt to parse each file and store the data in the dictionary
        try:
            # Extract second to last part of the file name
            sub_dir_name = file_name.rsplit('.', 2)[-2]
            # Parse Trodes file and store the data
            result[sub_dir_name] = read_trodes_extracted_data_file(os.path.join(dir_path, file_name))

        # Skip files that cause errors during parsing
        except Exception as e:
            print(f"Skipping file {file_name} due to error: {e}")
            continue

    return result

def organize_all_trodes_export(dir_path):
    """
    Organize Trodes files in subdirectories based on prefix and suffix of the subdirectory.
    The function creates a dictionary with subdirectory prefix and suffix as keys,
    and the output of `organize_trodes_files_by_suffix` as values.

    Args:
        dir_path (str): Path of the directory to process.

    Returns:
        dict: Nested dictionary with keys as subdirectory prefix and suffix and values 
        containing data obtained from the `organize_trodes_files_by_suffix` function.
    """
    result = defaultdict(dict)
    
    for sub_dir_name in os.listdir(dir_path):
        # Construct the full path to the subdirectory
        sub_dir_path = os.path.join(dir_path, sub_dir_name)
        # Process only if it's a directory
        if os.path.isdir(sub_dir_path):
            try:
                # Split the subdirectory name by dots to extract prefix and suffix
                sub_dir_name_parts = sub_dir_name.split('.')
                sub_dir_name_prefix = sub_dir_name_parts[0]
                sub_dir_name_suffix = sub_dir_name_parts[-1]
                # Organize the Trodes files in the subdirectory and store the results
                result[sub_dir_name_prefix][sub_dir_name_suffix] = organize_single_trodes_export(sub_dir_path)
            except Exception as e:
                print(f"Error processing subdirectory {sub_dir_path}: {e}")
                continue

    return result
def create_timestamps_dict(data_directory):
    """
    creates a dictionary of timestamps for each recording in the given directory 
    timestamps are zeroed to the start of the stream. timestamps are the sample number
    or sample index 
    two main elements of the dictionary are
    1. ephys recording timestamps
    2. video timestamps: another dictionary whose keys are the video names
       and values are timestamps for each frame of the video, i.e. the sample number
       for each frame of the video 

    Args(1):
        data_directory: str, path to data file 
            \\with\\doube\\slashes
    
    Returns (1):
        session_timestamps: dict, recordings to timestamps
            key: str, file name minus _merged.rec
            value: dict whose keys and values are
                'vid timestamps': {name of video, str: vid timestamps, np.array}
                'ephys timestamps': np.array of ephys sample timestamps

    """
    input_glob = data_directory + '\*.rec'
    all_session_files = glob.glob(input_glob)
    session_timestamps = {}
    for session_path in all_session_files:   
        vid_dict = {}
        session_basename = os.path.splitext(os.path.basename(session_path))[0]
        first_timestamp = None
        for root, dirs, files in os.walk(session_path):
            if os.path.basename(root).endswith('merged.raw'):
                for file in files:
                    if file.endswith('timestamps.dat'):
                        timestamps = os.path.join(root, file)
                        ephys_timestamp_array = read_trodes_extracted_data_file(timestamps)
                        ephys_timestamp = ephys_timestamp_array['data']
                        first_timestamp = ephys_timestamp_array['first_timestamp']  
            for file in files:
                if file.endswith('cameraHWSync'):
                    vid_timestamps_path = os.path.join(session_path, file)
                    timestamp_array = read_trodes_extracted_data_file(vid_timestamps_path)
                    vid_timestamps = timestamp_array["data"]["PosTimestamp"]
                    vid_dict[file] = vid_timestamps
        session_timestamps[session_basename] = {'vid timestamps': vid_dict, 
                                                'ephys timestamps':ephys_timestamp}
    
    return session_timestamps, first_timestamp

def find_closest_index(sorted_list=None, target=0):
    #leo's function that i am not sure i want to use at the moment 
    """
    Returns the index of the number in the sorted list that is closest to the target.

    This function performs a binary search on a sorted list to find the closest number to 
    a given target. If the target exists in the list, its index is returned. If not, the 
    function will return the index of the number that's closest to the target.

    Parameters:
    - sorted_list (list[int or float]): A sorted list of numbers.
    - target (int or float): The target number to find the closest value to.

    Returns:
    - int: The index of the closest number in the sorted list to the target. 
           If the sorted list is empty, returns None.

    Example:
    >>> sorted_nums = [1, 3, 5, 8, 10, 15, 18, 20, 24, 27, 30]
    >>> find_closest_index(sorted_nums, 6)
    2

    Note:
    The list should be sorted in ascending order.
    """
    
    if sorted_list is None:
        return None
    if target <= sorted_list[0]:
        return 0
    if target >= sorted_list[-1]:
        return len(sorted_list) - 1

    # Binary search
    left, right = 0, len(sorted_list) - 1
    while left <= right:
        mid = (left + right) // 2

        if sorted_list[mid] == target:
            return mid
        elif sorted_list[mid] < target:
            left = mid + 1
        else:
            right = mid - 1

    # After binary search, the target will be between sorted_list[right] and sorted_list[left]
    # We compare the two to see which one is closer to the target and return its index
    if abs(sorted_list[left] - target) < abs(sorted_list[right] - target):
        return left
    else:
        return right

In [ ]:
#the rest of leo's code that i am not sure what it does or how to deal with it 
    session_to_din_state_df[session_basename][os.path.basename(video_timestamps)] = session_to_din_state_df[session_basename]["time"].apply(lambda x: find_closest_index(sorted_list=timestamp_array, target=x))        
        
        # Find the maximum length of the arrays in the dictionary
        max_length = max(map(len, file_to_video_timestamps.values()))
        
        # Pad each array with NaN values to make them all the same length
        padded_data = {k: np.pad(v, (0, max_length - len(v)), mode='constant', constant_values=np.nan) for k, v in file_to_video_timestamps.items()}
        
        # Convert the padded data to a dataframe
        session_to_din_with_frames_df[session_basename] = pd.DataFrame(padded_data)
        session_to_din_with_frames_df[sesssion_basename].to_csv(os.path.join(OUTPUT_DIR, "{}.frame_to_timestamps.csv".format(session_basename)))
    except Exception as e: 
        print(e)